In [ ]:
from poselets import extract_poselets, biggest_poselet
from patches import patches_from_poselet
from configparser import ConfigParser
from skimage.io import imread, imshow, imsave
import mysql.connector as conn

In [ ]:
FEATURE_CODES = [
    24, 192,     # right & left elbow
    12, 96,      # right & left arm
    6, 34,       # right & left shoulder
    514, 4098,   # right & left torso
    1536, 12288, # right & left hip
    3072, 24576] # right & left thigh

In [ ]:
config = ConfigParser()
config.read_file(open('config.ini'))

In [ ]:
data_config = config['data']
samples_path = data_config.get('samples_path')

In [ ]:
database_config = config['database']
host = database_config.get('host')
name = database_config.get('name')
username = database_config.get('username')
password = database_config.get('password')

In [ ]:
db = conn.connect(host=host, database=name, user=username, passwd=password)

In [ ]:
cursor = db.cursor()

In [ ]:
cursor.execute('''
select samples.costumeFK, samples.file_name from samples
left join patches on patches.sampleFK = samples.sampleID
where samples.is_usable = 1 and patches.patchID is null
''')

In [ ]:
cls_images = [(c[0], imread(samples_path + c[1] + '.jpg')) for c in cursor.fetchall()]

In [ ]:
for cls, image in cls_images:
    all_keypoints = extract_poselets(image)
    if all_keypoints and len(all_keypoints):
        keypoints = biggest_poselet(all_keypoints)
        patches = patches_from_poselet(images[0], keypoints)
        for patch, feature_code in zip(patches, FEATURE_CODES):
            patch_path = '%s/%s.png' % (samples_path, cls)
            # uncomment to save patches
            #imsave(patch_path)
    else:
        print('No poselet detected', image)